# Bias Amplification by Inductive Transfer Learning

This notebook guides you through the process of bias amplification by inductive transfer learning. This demonstration uses MIDRC open A1 cheset X-ray image data as the example, and provides you with the instruction on how to train and deploy the model, as well as bias visualization.
The inductive transfer learning approach establishs a two-step transfer learning process to amplifies bias. In the first step, the AI model is trained to classify patients according to a subgroup attribute (e.g., patient sex). In the second step, the model is fine-tuned to the target clinical task. dditional control over the degree to which bias is promoted using this process can be obtained by altering the number of layers frozen during the second training step.

## Model Training

In this example you can directly uses the partitioned data sets (training, validation and testing) from the data_preprocessing example. All these three sets are equally stratified by patient sex (male and female), race (white and black) and COVID status (positive and negative). This demonstration takes patient sex as an example.

### First step training

To amplify bias by inductive transfer learning, the first step is to train the model to classify subgroup attribute (patient sex). You can run the following cell to train two separate sex classification models:
 - Model where "M" (male) is associated with model classification of “1”
 - Model where "F" (female) is associated with model classification of “1”

In this demonstration we uses *ResNet-18* as the example network architecture, and pre-trained weights trained from a contrastive self-supervised learning (CSL) approach and data from the CheXpert data. This weight file can be found under */example/* directory.

In [3]:
# first step training
main_dir = "/gpfs_projects/yuhang.zhang/OUT/2022_CXR/test/RAND_0"
task_list = ["M", "F"]
for task in task_list:
    %run ../src/utils/model_train.py -i "{main_dir}/train.csv" \
                                     -v "{main_dir}/validation.csv" \
                                     -o "{main_dir}/{task}" \
                                     -l "{main_dir}/{task}/run_log.log" \
                                     -c "checkpoint_csl.pth.tar" \
                                     -p "adam" \
                                     -g 0 \
                                     --pretrained_weights True \
                                     --train_task "{task}"

Full fine tuning selected
['patient_id', 'Path', 'M', 'F', 'White', 'Black', 'Yes', 'No']
['patient_id', 'Path', 'M', 'F', 'White', 'Black', 'Yes', 'No']
Training...
EPOCH	TR-AVG-LOSS	VD-AUC
> 0	0.52643		0.91901
> 1	0.26776		0.96004
> 2	0.17215		0.96730
> 3	0.09069		0.97279
> 4	0.06606		0.97341
> 5	0.05946		0.97458
> 6	0.04441		0.97463
> 7	0.04609		0.97450
> 8	0.04231		0.97473
> 9	0.03930		0.97479
> 10	0.03988		0.97474
> 11	0.03863		0.97491
> 12	0.03612		0.97491
> 13	0.03893		0.97476
> 14	0.04162		0.97470
Final epoch model saved to: /gpfs_projects/yuhang.zhang/OUT/2022_CXR/test/RAND_0//M/pytorch_last_epoch_model.onnx
Final epoch model saved to: /gpfs_projects/yuhang.zhang/OUT/2022_CXR/test/RAND_0//M/best_auc_model.onnx
END.


usage: model_train.py [-h] -i INPUT_TRAIN_FILE -v VALIDATION_FILE -o
                      OUTPUT_BASE_DIR [-d DCNN] [--freeze_up_to FREEZE_UP_TO]
                      [--pretrained_weights PRETRAINED_WEIGHTS]
                      [-f FINE_TUNING] [-m MOCO] [-u UPTO_FREEZE] -l LOG_PATH
                      -p OPTIMIZER [-b BATCH_SIZE] [-n NUM_EPOCHS]
                      [-t THREADS] [-r START_LEARNING_RATE] [-s STEP_DECAY]
                      [--SGDmomentum SGDMOMENTUM]
                      [--decay_every_N_epoch DECAY_EVERY_N_EPOCH]
                      [--decay_multiplier DECAY_MULTIPLIER]
                      [-e SAVE_EVERY_N_EPOCHS]
                      [--bsave_valid_results_at_epochs BSAVE_VALID_RESULTS_AT_EPOCHS]
                      [-g GPU_ID] [-c CUSTOM_CHECKPOINT_FILE]
                      [--random_state RANDOM_STATE] [--train_task TRAIN_TASK]
model_train.py: error: the following arguments are required: -p/--optimizer


SystemExit: 2

### Second step training

The second step is to fine-tune the model from step 1 to perform target clinical task. By running the following cell, you can fine-tune these two resulted models to predict COVID status, with different number of model layers being frozen. In this step, the same training/validation sets are used.

In [ ]:
# second step training
frozen_layers = [1, 14]
for task in task_list:
    for n in frozen_layers:
        %run ../src/utils/model_train.py -i "{main_dir}/train.csv" \
                                         -v "{main_dir}/validation.csv" \
                                         -o "{main_dir}/{task}/{n}_frozen_layer/" \
                                         -l "{main_dir}/{task}/{n}_frozen_layer/run_log.log" \
                                         -c "{main_dir}/{task}/checkpoint__last.pth.tar" \
                                         -g 0 \
                                         --pretrained_weights True

### Baseline model training

To show the degree to which bias is amplified by this approach, a baseline model is required to present baseline bias. You can run the following cell to train the baseline. To make fair comparison, the baseline uses the same model architecture and pre-trained weights (from CSL approach), as well as the same training/validation sets. However, it will skip the first step training, and directly train to perform COVID status prediction.

In [ ]:
# baseline training
%run ../src/utils/model_train.py     -i "{main_dir}/train.csv" \
                                     -v "{main_dir}/validation.csv" \
                                     -o "{main_dir}/baseline" \
                                     -l "{main_dir}/baseline/run_log.log" \
                                     -c "checkpoint_csl.pth.tar" \
                                     -g 0 \
                                     --pretrained_weights True

## Model Inference

After model training is done, you can deploy the models on the independent testing set by running the inference code below. The inference code will save prediction scores as *results__.tsv* files under the same directory.

## Bias Visualization

After inference, you can analyze the model bias by running the following code. The analysis code here will calculate the subgroup **predicted prevalence** and **AUROC** , and plot these measurements with respect to training disease prevalence differences between two subgroups.